In [ ]:
pip install yfinance


In [120]:
pip install scikit-learn


     |████████████████████████████████| 11.1 MB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 46.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:

"""model = Sequential([
    LSTM(1000, return_sequences=True, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.4),
    LSTM(50, return_sequences=False),
    Dense(1)  # Output layer for predicting stock_close
])"""

"""
model = Sequential([
    LSTM(1000, return_sequences=False, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.4),
    Dense(1)  # Output layer for predicting stock_close
]) sequencia 10 prever 7
"""


model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.4),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(1)  # Output layer for predicting stock_close
])
#model.compile(optimizer='adam', loss='mean_squared_error')
#model.fit(X_train, y_train, batch_size=32, epochs=64, verbose=1)
#Melhor R2, desejado prever 7, uso sequncial 5




# Train the model on the entire dataset


# Make predictions





In [5]:

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.regularizers import l1, l2, l1_l2
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from datetime import datetime
import pandas as pd
import os

2024-12-03 16:29:37.118918: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 16:29:37.694549: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 16:29:37.699438: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 16:29:40.964937: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
symbol_usdt_to_brl = 'BRL=X'
symbol_acao = 'acao.SA'
start_date = '2021-02-22'
end_date = '2024-11-20'
start_date_test_model = '2024-11-21'
end_date_test_model = '2024-11-29'
path_files_original = "originalFiles"
path_files_process = "preprocessFiles"

In [7]:
def ajusting_tables_TIOc1(df):
    df_a = df
    try:
        df_a['Data'] = pd.to_datetime(df_a['Data'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')
    except:
        print("")
    df_a = df_a.iloc[::-1].reset_index(drop=True)
    df_a['Data'] = pd.to_datetime(df_a['Data'], format='%Y-%m-%d')

    df_a['Último'] = df_a['Último'].str.replace(',', '.').astype(float)
    df_a['Abertura'] = df_a['Abertura'].str.replace(',', '.').astype(float)
    df_a['Máxima'] = df_a['Máxima'].str.replace(',', '.').astype(float)
    df_a['Mínima'] = df_a['Mínima'].str.replace(',', '.').astype(float)

    #Pegando apenas colunas necessárias
    df_a = df_a[["Data","Último","Abertura","Máxima","Mínima"]]

    #Renomeando Colunas
    df_a.rename(columns={"Data": "Date","Último": "close_TIOc1", "Máxima": "high_TIOc1", "Mínima": "low_TIOc1", "Abertura": "open_TIOc1"}, inplace=True)
    return df_a

def ajusting_tables_dolar(df):
    df_a = df

    df_a = df_a[["Price","High","Low","Open","Close"]]

    df_a = df_a.drop([0, 1], axis=0)

    df_a.rename(columns={"Price":"Date","Close": "close_dolar", "High": "high_dolar", "Low": "low_dolar", "Open": "open_dolar"}, inplace=True)
    df_a = df_a.reset_index(drop=True)
    return df_a

def ajusting_tables_action(df):
    df_b = df

    df_b = df_b[["Price","High","Low","Open","Volume","Close"]]

    df_b = df_b.drop([0, 1], axis=0)

    df_b.rename(columns={"Price":"Date","Close": "close_acao", "High": "high_acao", "Low": "low_acao", "Open": "open_acao", "Volume": "volume_acao"}, inplace=True)
    df_b = df_b.reset_index(drop=True)

    return df_b

def ajust_dates(df_a, df_b, df_c):
    
    #Ajustando o shape, por conta de feriados e a bolsa não funcionar, é necessário deixa-los com o mesmo tamanho
    df_a['Date'] = pd.to_datetime(df_a['Date'], format='%Y-%m-%d')
    df_b['Date'] = pd.to_datetime(df_b['Date'], format='%Y-%m-%d')
    df_c['Date'] = pd.to_datetime(df_c['Date'], format='%Y-%m-%d')

    common_dates_b_c = set(df_b['Date']).intersection(df_c['Date'])

    # Filtre ambos os DataFrames para manter apenas as linhas com datas comuns
    df_b = df_b[df_b['Date'].isin(common_dates_b_c)]
    df_c = df_c[df_c['Date'].isin(common_dates_b_c)]

    # (Opcional) Resetar os índices, já que as linhas foram filtradas
    df_b.reset_index(drop=True, inplace=True)
    df_c.reset_index(drop=True, inplace=True)
    
    common_dates_a_c = set(df_a['Date']).intersection(df_c['Date'])
    
    df_a = df_a[df_a['Date'].isin(common_dates_a_c)]
    df_c = df_c[df_c['Date'].isin(common_dates_a_c)]

    df_a.reset_index(drop=True, inplace=True)
    df_c.reset_index(drop=True, inplace=True)
    
    #Jutando dataframes
    midle = pd.merge(df_a,df_c, on="Date", how="inner")

    df_to_process = pd.merge(midle,df_b, on="Date", how="inner").drop(columns=['Date'])
    return df_to_process

In [8]:
df_acao_origin = pd.read_csv(path_files_original + "/acao.csv")
df_acao_origin_test_model_origin = pd.read_csv(path_files_original + "/acao_test_model.csv")
df_usdt_to_brl_origin = pd.read_csv(path_files_original + "/usdt2brl.csv")
df_iron_ore_TIOc1_origin = pd.read_csv(path_files_original + "/Contrato Futuro Minério de ferro refinado TIOc1.csv")
df_iron_ore_DCIOF5_origin = pd.read_csv(path_files_original + "/Contrato Futuro Minério de ferro refinado DCIOF5.csv")


df_dolar = ajusting_tables_dolar(df_usdt_to_brl_origin)
df_acao = ajusting_tables_action(df_acao_origin)
df_acao_test_model = ajusting_tables_action(df_acao_origin_test_model_origin)
df_tioc1 = ajusting_tables_TIOc1(df_iron_ore_TIOc1_origin)
df_process = ajust_dates(df_dolar, df_acao, df_tioc1)

In [49]:
import pandas as pd

# Função para adicionar dias úteis
def adicionar_dias_uteis(data_inicial, dias_para_adicionar):
    # Converte a data para o formato pandas datetime
    data_inicial = pd.to_datetime(data_inicial)
    
    # Adiciona os dias úteis à data inicial
    dias_adicionados = 0
    while dias_adicionados < dias_para_adicionar:
        data_inicial += pd.Timedelta(days=1)
        if data_inicial.weekday() < 5:  # Se for um dia útil (segunda a sexta)
            dias_adicionados += 1
    
    return data_inicial

# Exemplo de uso
data_inicial = '2024-11-21'
dias_para_adicionar = 5

data_final = adicionar_dias_uteis(data_inicial, dias_para_adicionar)
print("Data final ajustada (após considerar apenas dias úteis):", data_final)

pd.DataFrame(pd.date_range(start=data_final, periods=5, freq='B'), columns=["Date"])

Data final ajustada (após considerar apenas dias úteis): 1970-01-08 00:00:00.000001992


,Date
0,1970-01-08 00:00:00.000001992
1,1970-01-09 00:00:00.000001992
2,1970-01-12 00:00:00.000001992
3,1970-01-13 00:00:00.000001992
4,1970-01-14 00:00:00.000001992


In [44]:
df_process.shape[0]

742

In [ ]:
target = "close_acao"
sequence_length = 10
df_process = df_process
features = df_process.columns[0:-1].to_list()
scaler = MinMaxScaler()
value_train_split = 0.8
batch_size = 32
epochs = 64
wish_perid_to_predict = 7

scaled_data = scaler.fit_transform(df_process[features + [target]])

X, y = [], []
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i, :-1])  
    y.append(scaled_data[i, -1]) 

X = np.array(X)
y = np.array(y)

train_size = int(len(X) * value_train_split )
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
            Dropout(0.4),
            LSTM(50, return_sequences=False),
            Dropout(0.2),
            Dense(1)  # Output layer for predicting stock_close
        ])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=32, epochs=32, verbose=1)
predictions = model.predict(X_test)

rescaled_predictions = scaler.inverse_transform(
    np.hstack((np.zeros((len(predictions), len(features))), predictions.reshape(-1, 1)))
)[:, -1]  # Rescale only the predicted close price

# Rescale actual values
actual_close = scaler.inverse_transform(
    np.hstack((np.zeros((len(y_test), len(features))), y_test.reshape(-1, 1)))
)[:, -1]

In [ ]:
import yfinance as yf


start_date_test_model = '2024-11-21'
end_date_test_model = '2024-11-28'
wish_perid_to_predict = 7
sequence_length = 3

#Ultimo sequence_length valores usados para o treinamento
last_sequence = scaled_data[-sequence_length:, :-1] #Sem a coluna target
future_predictions = []

for i in range(wish_perid_to_predict):
    last_sequence_reshaped = np.expand_dims(last_sequence, axis=0)

    next_prediction = model.predict(last_sequence_reshaped)[0, 0]  # Saída única

    future_predictions.append(next_prediction)


    next_sequence = np.append(last_sequence[1:], [[*last_sequence[-1, :-1], next_prediction]], axis=0)

    last_sequence = next_sequence



# Reverter a escala das previsões para a escala original
rescaled_future_predictions = scaler.inverse_transform(
    np.hstack((np.zeros((len(future_predictions), len(features))), np.array(future_predictions).reshape(-1, 1)))
)[:, -1]

prediction_dates = pd.date_range(start='2024-11-21', periods=wish_perid_to_predict, freq='B')
filter_dates_hist = df_acao[df_acao["Date"] >= "2024-10-01"]

combined_dates = pd.concat([filter_dates_hist['Date'],df_acao_test_model["Date"]])

values_actual = np.concatenate([actual_close[-filter_dates_hist[target].count():], df_acao_test_model[target].to_numpy()])

# Plotar os resultados
plt.figure(figsize=(14, 7))

plt.plot(combined_dates, values_actual, label='Dados Históricos', color='blue', marker='o')

plt.plot(filter_dates_hist['Date'], rescaled_predictions[-filter_dates_hist[target].count():], label='Dados Previstos', color='red', marker='o')

plt.plot(prediction_dates, rescaled_future_predictions, label='Dados Previstos', color='orange', marker='o')
plt.title('Previsões de Preço de Fechamento')
plt.xlabel('Datas')
plt.ylabel('Preço de Fechamento')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import yfinance as yf


start_date_test_model = '2024-11-21'
end_date_test_model = '2024-11-28'
wish_perid_to_predict = 7
sequence_length = 3

#Ultimo sequence_length valores usados para o treinamento
last_sequence = scaled_data[-sequence_length:, :-1] #Sem a coluna target
future_predictions = []

for i in range(wish_perid_to_predict):
    last_sequence_reshaped = np.expand_dims(last_sequence, axis=0)

    next_prediction = model.predict(last_sequence_reshaped)[0, 0]  # Saída única

    future_predictions.append(next_prediction)


    next_sequence = np.append(last_sequence[1:], [[*last_sequence[-1, :-1], next_prediction]], axis=0)

    last_sequence = next_sequence



# Reverter a escala das previsões para a escala original
rescaled_future_predictions = scaler.inverse_transform(
    np.hstack((np.zeros((len(future_predictions), len(features))), np.array(future_predictions).reshape(-1, 1)))
)[:, -1]


date_predict = pd.DataFrame(pd.date_range(start=df_acao_test_model["Date"][0], periods=wish_perid_to_predict, freq='B'), columns=["Date"])
# Plotar os resultados
plt.figure(figsize=(14, 7))
plt.plot(pd.to_datetime(df_acao_test_model["Date"]), df_acao_test_model[target], label='Dados Históricos', color='blue', marker='o')
plt.plot(date_predict, rescaled_future_predictions, label='Dados Previstos', color='orange', marker='o')
plt.title('Previsões de Preço de Fechamento')
plt.xlabel('Datas')
plt.ylabel('Preço de Fechamento')
plt.legend()
plt.grid(True)
plt.show()

In [3]:
import plotly.graph_objects as go

data = {
    'df_acao_test_model': [
        {'Date': '2024-11-21', 'close_acao': 5.110000133514404},
        {'Date': '2024-11-22', 'close_acao': 5.190000057220459},
        {'Date': '2024-11-25', 'close_acao': 5.21999979019165},
        {'Date': '2024-11-26', 'close_acao': 5.170000076293945},
        {'Date': '2024-11-27', 'close_acao': 5.199999809265137},
        {'Date': '2024-11-28', 'close_acao': 5.130000114440918}
    ],
    'date_predict': ['2024-11-21', '2024-11-22', '2024-11-25', '2024-11-26', '2024-11-27'],
    'predict_sample_lstm': [5.2537069959272165, 5.264688588487188, 5.262257691854784, 5.289070754079574, 5.289379155648134],
    'predict_lstm_bidirecional': [5.088874301638454, 5.119208305414147, 5.135722258954927, 5.1474337163126185, 5.14820216677117],
    'predict_lstm_attention': [5.0401267098212506, 5.056832031216671, 5.052904663495609, 5.080409430434428, 5.080752587469546],
    'predict_lstm_cnn': [5.054553207482839, 5.065512670031452, 5.066775073692269, 4.996003297364836, 4.995404510326707],
    'predict_lstm_bi_atten_cnn': [5.252298051870667, 5.246177259562173, 5.259210701380495, 5.221314333658909, 5.184980827060058]
}

# Convertendo o JSON em DataFrames
df_acao_test_model = pd.DataFrame(data['df_acao_test_model'])
df_acao_test_model['Date'] = pd.to_datetime(df_acao_test_model['Date'])

date_predict = pd.to_datetime(data['date_predict'])

# Criando o gráfico interativo
fig = go.Figure()

# Adicionando os dados históricos
fig.add_trace(go.Scatter(
    x=df_acao_test_model['Date'],
    y=df_acao_test_model['close_acao'],
    mode='lines+markers',
    name='Dados Históricos',
    line=dict(color='blue')
))

# Adicionando previsões
fig.add_trace(go.Scatter(
    x=date_predict,
    y=data['predict_sample_lstm'],
    mode='lines+markers',
    name='Previsão LSTM',
    line=dict(color='orange')
))

fig.add_trace(go.Scatter(
    x=date_predict,
    y=data['predict_lstm_bidirecional'],
    mode='lines+markers',
    name='Previsão LSTM Bidirecional',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=date_predict,
    y=data['predict_lstm_attention'],
    mode='lines+markers',
    name='Previsão LSTM + Attention',
    line=dict(color='gray')
))

fig.add_trace(go.Scatter(
    x=date_predict,
    y=data['predict_lstm_cnn'],
    mode='lines+markers',
    name='Previsão LSTM + CNN',
    line=dict(color='green')
))

fig.add_trace(go.Scatter(
    x=date_predict,
    y=data['predict_lstm_bi_atten_cnn'],
    mode='lines+markers',
    name='Previsão LSTM Bi-Attention + CNN',
    line=dict(color='pink')
))

# Customizando o layout
fig.update_layout(
    title='Previsões de Preço de Fechamento',
    xaxis_title='Datas',
    yaxis_title='Preço de Fechamento',
    legend_title='Modelos',
    template='plotly_white'
)

# Exibindo o gráfico
fig.show()